<a href="https://colab.research.google.com/github/SolSolstice/APIs-Challenge/blob/main/challenge_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os # allow for pyspark / hadoop / java instance components to be installed into the runtime
# set up a reference to the pyspark instance - stable version - 3.3.2
spark_version = 'spark-3.3.2'
# use os.environ function to reference 'SPARK_VERSION' property
os.environ['SPARK_VERSION'] = spark_version
# Install the spark and java components
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# set the runtime environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64" # sets the path to the Java Virtual Machine (Java Development Kit)
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# start a spark session by importing and using findspark
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:

# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homeDF = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True,ignoreLeadingWhiteSpace=True)
homeDF.show(10)



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 2. Create a temporary view of the DataFrame.
homeDF.createOrReplaceTempView('home_data')



In [23]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_4bed = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) FROM home_data WHERE bedrooms == 4 GROUP BY YEAR(date)")
avg_4bed.show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
+--------------------+----------+



In [24]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_3bed3bath = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) FROM home_data WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY Year(date)")
avg_3bed3bath.show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           292725.69|      2022|
|           287287.82|      2019|
|           294204.16|      2020|
|           294211.46|      2021|
+--------------------+----------+



In [25]:
#double-checking column names..
homeDF.distinct()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

personal note ~

How to break long line of code to next line but remain part of same statement

source ~ https://www.interviewqs.com/ddi-code-snippets/break-long-line-python

In [36]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_2floor_3bed3bath_2ksqF = spark.sql("SELECT ROUND(AVG(price),2), date_built FROM home_data WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND\
 sqft_living >= 2000 GROUP BY date_built ORDER BY date_built DESC")

avg_2floor_3bed3bath_2ksqF.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



personal note ~

*The HAVING clause is used to filter the results produced by GROUP BY based on the specified condition. It is often used in conjunction with a GROUP BY clause.*

source ~ https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-having.html

In [38]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_data GROUP BY view HAVING ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 1.030797004699707 seconds ---


In [42]:
# 7. Cache the the temporary table home_sales.

# assuming this meant to say home_data instead of home_sales
spark.sql("cache table home_data")


DataFrame[]

In [43]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_data')

True

In [44]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_data GROUP BY view HAVING ROUND(AVG(price),2) < 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 1.1028509140014648 seconds ---


In [45]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homeDF.write.parquet("home_parquet",mode="overwrite")

In [47]:
# 11. Read the parquet formatted data.
parquet_homeDF = spark.read.parquet("home_parquet")

In [50]:
# 12. Create a temporary table for the parquet data.
parquet_homeDF.createOrReplaceTempView("parquet_temp_home")

In [51]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) FROM parquet_temp_home GROUP BY view HAVING ROUND(AVG(price),2) < 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 1.0213556289672852 seconds ---


In [52]:
# 14. Uncache the home_sales temporary table.
spark.sql("Uncache table home_data")

DataFrame[]

In [53]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_data')


False